In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.layers import Dropout
from keras.applications import MobileNet
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.models import load_model
from keras.preprocessing.image import img_to_array

Using TensorFlow backend.


In [2]:
# base_model = VGG19(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [3]:
name_base_model = "VGG16"
base_model=VGG16(weights='imagenet',include_top=False) 
#imports the mobilenet model and discards the last 1000 neuron layer.

In [4]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dropout(0.25)(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [5]:
model=Model(inputs=base_model.input,outputs=preds)

In [6]:
# for i,layer in enumerate(model.layers):
#   print(i,layer.name)

In [7]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [8]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('data/training-data',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)
test_generator = train_datagen.flow_from_directory('data/testing-data',
                                                  target_size = (224,224),
                                                  color_mode='rgb',
                                                  batch_size=12,
                                                  class_mode='categorical',
                                                  shuffle=True)

Found 2000 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [9]:
# Specify parameters
lr = 0.1
epochs = 10

In [ ]:
Adam = keras.optimizers.Adam(lr=lr)
model.compile(optimizer=Adam,loss='binary_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be cbinary_crossentropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
step_size_test = test_generator.n//test_generator.batch_size
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=step_size_train,
                              validation_data=test_generator,
                              validation_steps=step_size_test,
                               epochs=epochs)

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Epoch 1/10
62/62 [==============================] - 1305s 21s/step - loss: 7.6223 - acc: 0.5186 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 2/10
62/62 [==============================] - 1317s 21s/step - loss: 7.9664 - acc: 0.5030 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 3/10
22/62 [=========>....................] - ETA: 13:29 - loss: 7.5369 - acc: 0.5298

In [12]:
# Scalar test loss (if the model has a single output and no metrics) or 
#list of scalars (if the model has multiple outputs and/or metrics).
#The attribute model.metrics_names will give you the display labels for the scalar outputs.
model.evaluate_generator(generator=test_generator,steps=step_size_test)

[5.431534838676453, 0.558333334326744]

In [23]:
test_generator.reset()
pred = model.predict_generator(test_generator, steps=step_size_test, verbose=1)

10/10 [==============================] - 9s 906ms/step


In [24]:
predicted_class_indices=np.argmax(pred,axis=1)
y_true = test_generator.classes

In [32]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, predicted_class_indices))

              precision    recall  f1-score   support

           0       0.50      0.97      0.66        60
           1       0.60      0.05      0.09        60

   micro avg       0.51      0.51      0.51       120
   macro avg       0.55      0.51      0.38       120
weighted avg       0.55      0.51      0.38       120



In [15]:
model.save("{}-{}-{}-adam.h5".format(name_base_model, epochs, lr))

In [ ]:
#testing (Code below is only for testing and is not serving any purpose.)

In [28]:
# from PIL import Image
# Image.imread("tb.jpg")
# img = Image.open("tb.jpg")
# img = img_to_array(img)
# image = np.expand_dims(img, axis=0)
# model.predict(image)

In [1]:
# model.summary()

In [2]:
# model.get_weights()

In [3]:
# base_model.get_weights()

In [2]:
# model = load_model("2000-img-adam.h5")

In [9]:
model.predictions


AttributeError: 'Model' object has no attribute 'predictions'